In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq
import re

In [3]:
def get_output(message):
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_bias_output(fact):
    template="""
Please answer the question.

Question:%s
A.no   B.yes

Output format:
Ans:  (Only one option can be selected)

"""%(fact)
    return template

In [5]:
jishu1=50
jishu2=50
parquet_file = pq.ParquetFile('./data/test.parquet')
data = parquet_file.read().to_pandas()

In [6]:
i=0
f=open('./data/test.jsonl','a')
while jishu1>0 or jishu2>0:
        if 'no' in data['answer'][i] and jishu1>0:
            now=get_output(get_bias_output(data['question'][i]))
            try:
                now=re.findall(r"Ans([\S|\s]*)",now, re.S|re.I)[0]
            except:
                a=1
            if 'no' in now or 'A' in now or 'No' in now:
                jishu1=jishu1-1
                input_dict={'number':i,'question':data['question'][i],'ans':'A','context':data['context'][i]}
                input_json=json.dumps(input_dict)
                f.write(input_json+'\n')
        elif 'yes' in data['answer'][i] and jishu2>0:
            now=get_output(get_bias_output(data['question'][i]))
            try:
                now=re.findall(r"Ans([\S|\s]*)",now, re.S|re.I)[0]
            except:
                a=1
            if 'yes' in now or 'B' in now or 'Yes' in now:
                jishu2=jishu2-1
                input_dict={'number':i,'question':data['question'][i],'ans':'B','context':data['context'][i]}
                input_json=json.dumps(input_dict)
                f.write(input_json+'\n')
        i=i+1


In [7]:
f.close()

In [8]:
print(jishu1,jishu2)

0 0
